# Sequence to Sequence (a.k.a. seq2seq)

**학습목표**
* Encoder Decoder 구조를 이해하고 구현할 줄 안다.
* Seq2Seq에 필요한 전처리를 이해한다.
* **데이터 부족**과, **긴 문장**을 겪어본다.

![이런거](https://raw.githubusercontent.com/KerasKorea/KEKOxTutorial/master/media/28_1.png)
---------------------------------
edu.rayleigh@gmail.com
Special Thanks to : 숙번님 ( [봉수골 개발자 이선비](https://www.youtube.com/channel/UCOAyyrvi7tnCAz7RhH98QCQ) )

In [1]:
!wget http://www.manythings.org/anki/kor-eng.zip

--2023-03-31 05:11:00--  http://www.manythings.org/anki/kor-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 224550 (219K) [application/zip]
Saving to: ‘kor-eng.zip’

kor-eng.zip         100%[===================>] 219.29K   386KB/s    in 0.6s    

2023-03-31 05:11:01 (386 KB/s) - ‘kor-eng.zip’ saved [224550/224550]



In [2]:
import zipfile
kor_eng = zipfile.ZipFile('kor-eng.zip')
kor_eng.extractall()
kor_eng.close()

In [3]:
import pandas as pd
temp = pd.read_table('kor.txt', names=['Eng', 'Kor', 'license'])
temp.shape

(5749, 3)

In [4]:
temp.head()

,Eng,Kor,license
0,Go.,가.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Hi.,안녕.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
2,Run!,뛰어!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
3,Run.,뛰어.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
4,Who?,누구?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [5]:
eng_sent = temp['Eng'].tolist()
kor_sent = temp['Kor'].tolist()

In [6]:
print(eng_sent[1000])
print(kor_sent[1000])

My cat is black.
내 고양이는 검은색 고양이야.


# 데이터 준비
0. 단어와 구두점 사이 공백 만들기
1. sos 와 eos
1. tokenizing, idx_seq, padding

## 0. 단어와 구두점 사이 공백 만들기


In [7]:
import unicodedata
import re
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')
def eng_preprocessor(sent):
    # 위에서 구현한 함수를 내부적으로 호출
    sent = unicode_to_ascii(sent.lower())

    # 단어와 구두점 사이에 공백을 만듭니다.
    # Ex) "he is a boy." => "he is a boy ."
    sent = re.sub(r"([?.!,'¿])", r" \1 ", sent)

    # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환합니다.
    sent = re.sub(r"[^a-zA-Z!.?']+", r" ", sent)

    sent = re.sub(r"\s+", " ", sent)
    return sent

def kor_preprocessor(sent):
    # 위에서 구현한 함수를 내부적으로 호출
    sent = unicode_to_ascii(sent.lower())

    # 단어와 구두점 사이에 공백을 만듭니다.
    # Ex) "he is a boy." => "he is a boy ."
    sent = re.sub(r"([?.!,'¿])", r" \1 ", sent)

    sent = re.sub(r"\s+", " ", sent)
    return sent

In [11]:
eng_preprocessor("I'm just a poor boy.")

"i ' m just a poor boy . "

In [9]:

eng_sent = [ eng_preprocessor(sent) for sent in eng_sent ]
kor_sent = [ kor_preprocessor(sent) for sent in kor_sent ]

In [10]:

print(eng_sent[1000])
print(kor_sent[1000])

my cat is black . 
내 고양이는 검은색 고양이야 . 


## 1. sos 와 eos
1. sos : start of speech
2. eos : end of speech

In [12]:
######################
### Your Code here ###
######################

## 영어 문장 전 후에 <sos>와 <eos>를 추가할 것
## 띄어쓰기 주의!
eng_sent = [f'<sos> {eng} <eos>' for eng in eng_sent]
eng_sent[1000]

'<sos> my cat is black .  <eos>'

## 2. Tokenizing, idx_seq, padding

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [19]:
######################
### Your Code here ###
######################

# Tokenizing    # 한국어는 lower = False
tokenizer_kr = Tokenizer(filters="", lower=False)
tokenizer_kr.fit_on_texts(kor_sent)

tokenizer_en = Tokenizer(filters="", lower=True)
tokenizer_en.fit_on_texts(eng_sent)

In [20]:
######################
### Your Code here ###
######################

# Index Sequence
eng_seq = tokenizer_en.texts_to_sequences(eng_sent)
kor_seq = tokenizer_kr.texts_to_sequences(kor_sent)

print(eng_seq[1000])
print(kor_seq[1000])

[1, 24, 138, 11, 560, 3, 2]
[12, 257, 1109, 1087, 1]


In [21]:
######################
### Your Code here ###
######################
## 최대 문장 길이에 맞춰지도록 할 것.
# padding
eng_pad = pad_sequences(eng_seq)
kor_pad = pad_sequences(kor_seq)

print(eng_pad.shape)
print(kor_pad.shape)

(5749, 104)
(5749, 95)


In [22]:
# tokenizer에서 0 index가 구성되어있지 않지만, 
# pad_sequence에서 pad의 의미로 0을 사용하고 있어서, 전체 사이즈를 구할 때, +1을 해준다.

eng_vocab_size = len(tokenizer_en.word_index) + 1
kor_vocab_size = len(tokenizer_kr.word_index) + 1
print("영어 단어 집합의 크기: {:d}\n한국어 단어 집합의 크기: {:d}".format(eng_vocab_size, kor_vocab_size))

영어 단어 집합의 크기: 3079
한국어 단어 집합의 크기: 7842


# 모델링!

1. 모든 임베딩 레이어는 128개 차원으로 구성.
2. 인코더도 디코더도 GRU, 히든스테이트 512로 구성.
3. 디코더의 GRU 뒤에는 Fully Conneceted layer 사용. 노드 512개
4. 적절한 아웃풋레이어
    * 매 시점, 가장 적절한 단어가 무엇일지 분류 한다고 생각하면 됨!

In [25]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, GRU, Dense

In [33]:
######################
### Your Code here ###
######################

# 혹시 이미 그려둔 그래프가 있다면 날려줘!
tf.keras.backend.clear_session()

# 한국어 단어 집합의 크기 : 5551, (50000, 95)
# 영어 단어 집합의 크기 : 2484, (50000, 104)


# Encoder
enc_X = Input(shape=[kor_pad.shape[1]])
enc_E = Embedding(kor_vocab_size, 128)(enc_X)
enc_S_full, enc_S = GRU(512, return_sequences=True, return_state=True)(enc_E)
# Decoder
dec_X = Input(shape=[eng_pad.shape[1]-1]) # eos 제외
dec_E = Embedding(eng_vocab_size, 128)(dec_X)
dec_H = GRU(512, return_sequences=True)(dec_E, initial_state=enc_S)
# Model
dec_H = Dense(512, activation='swish')(dec_H)
dec_Y = Dense(eng_vocab_size, activation='softmax')(dec_H)

model = tf.keras.models.Model([enc_X, dec_X], dec_Y)

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 95)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 103)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 95, 128)      1003776     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 103, 128)     394112      ['input_2[0][0]']                
                                                                                              

In [34]:
######################
### Your Code here ###
######################
## 학습 시킬 것!
model.fit([kor_pad, eng_pad[:,:-1]], eng_pad[:,1:],
          shuffle=True,
          batch_size=128, epochs=10)

Epoch 1/10
45/45 [==============================] - 26s 355ms/step - loss: 1.7472 - accuracy: 0.8923
Epoch 2/10
45/45 [==============================] - 9s 199ms/step - loss: 0.5524 - accuracy: 0.9147
Epoch 3/10
45/45 [==============================] - 10s 229ms/step - loss: 0.4415 - accuracy: 0.9231
Epoch 4/10
45/45 [==============================] - 8s 174ms/step - loss: 0.4000 - accuracy: 0.9326
Epoch 5/10
45/45 [==============================] - 9s 190ms/step - loss: 0.3978 - accuracy: 0.9322
Epoch 6/10
45/45 [==============================] - 7s 164ms/step - loss: 0.3726 - accuracy: 0.9350
Epoch 7/10
45/45 [==============================] - 6s 133ms/step - loss: 0.3568 - accuracy: 0.9356
Epoch 8/10
45/45 [==============================] - 7s 150ms/step - loss: 0.3632 - accuracy: 0.9352
Epoch 9/10
45/45 [==============================] - 7s 147ms/step - loss: 0.3446 - accuracy: 0.9369
Epoch 10/10
45/45 [==============================] - 6s 137ms/step - loss: 0.3670 - accuracy: 0.93

In [35]:
import numpy as np

# 한국어 단어 집합의 크기 : 5551, (50000, 95)
# 영어 단어 집합의 크기 : 2484, (50000, 104)

def translate(kor):
    # eng => index => pad
    kor_seq = tokenizer_kr.texts_to_sequences([kor])
    kor_pad = tf.keras.preprocessing.sequence.pad_sequences(kor_seq, maxlen=95)

    eng = []
    for n in range(104-1):
        # kor => index => pad
        eng_seq = tokenizer_en.texts_to_sequences([['<sos>'] + eng])
        eng_pad = tf.keras.preprocessing.sequence.pad_sequences(eng_seq, maxlen=104-1)
        eng_next = model.predict([kor_pad, eng_pad])

        # onehot -> index -> word
        eng = [tokenizer_en.index_word[i] for i in np.argmax(eng_next[0], axis=1) if i != 0]
        # 번역된 word 선택
        eng = eng[:n+1]
        
        if eng[-1] == '<eos>':
            break

    return eng

In [37]:
import random

# 랜덤 10개
indices = list(range(3648))
random.shuffle(indices)

for n in indices[:10]:
    print(f"한국어: {kor_sent[n]}\n영어: {eng_sent[n]}")
    print(f"번역: {' '.join(translate(kor_sent[n])[:-1])}")
    print()

한국어: 톰은 그의 턱수염을 땋았다 . 
영어: <sos> tom has braided his beard .  <eos>
1/1 [==============================] - 0s 27ms/step
번역: i ' m ' t ' t ' t the very good .

한국어: 나는 그것을 허락하지 않을거야 . 
영어: <sos> i won ' t permit it .  <eos>
1/1 [==============================] - 0s 25ms/step
번역: i ' m ' t ' t ' t the very good .

한국어: 톰은 정말 무례하다 . 
영어: <sos> tom is so rude .  <eos>
1/1 [==============================] - 0s 26ms/step
번역: i ' m ' t ' t ' t the very good .

한국어: 여권 있어 ? 
영어: <sos> do you have your passport ?  <eos>
1/1 [==============================] - 0s 25ms/step
번역: i ' m ' t ' t ' t the very good .

한국어: 경주에서 이겼습니까 ? 
영어: <sos> did you win the race ?  <eos>
1/1 [==============================] - 0s 26ms/step
번역: i ' m ' t ' t ' t the very good .

한국어: 이것은 내 사전이다 . 
영어: <sos> this is my dictionary .  <eos>
1/1 [==============================] - 0s 27ms/step
번역: i ' m ' t ' t ' t the very good .

한국어: 여기가 ᄋ